In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Import MCP modules
from db_connection import DatabaseConnection
from stored_procedures import StoredProcedures
from report_generator import ReportGenerator
from config import ResortConfig, DatabaseConfig
from db_queries import execute_query_to_dataframe, get_revenue_query

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ All modules imported successfully!")


---

## ⚙️ Configuration

Set your report parameters here. Customize these values based on your needs.

### Available Resorts:
- **Snowbowl**
- **Purgatory** (Group 46)
- **Brian Head**
- **Lee Canyon**
- **Nordic Valley**
- **Sipapu**
- **Willamette**

### Purgatory Group Numbers:
- `46` - *PURGATORY
- `54` - *HESPERUS
- `59` - *SNOWCAT
- `67` - *SPIDER MOUNTAIN
- `70` - *DMMA
- `71` - *WILLAMETTE
- `-1` - All groups


In [ ]:
# ========== CONFIGURATION ==========

RESORT = "Purgatory"           # Resort name
DATABASE = "Purgatory"         # Database name (usually same as resort)
GROUP_NO = 46                  # Group number (46 for Purgatory, -1 for all)
DAYS_BACK = 30                 # Number of days to look back
OUTPUT_DIR = "reports"         # Output directory for CSV files

# Calculate date range
date_end = datetime.now()
date_ini = date_end - timedelta(days=DAYS_BACK)

print(f"📊 Configuration:")
print(f"   Resort: {RESORT}")
print(f"   Database: {DATABASE}")
print(f"   Group Number: {GROUP_NO}")
print(f"   Date Range: {date_ini.strftime('%Y-%m-%d')} to {date_end.strftime('%Y-%m-%d')}")
print(f"   Output Directory: {OUTPUT_DIR}")


In [ ]:
# Initialize report generator
generator = ReportGenerator(OUTPUT_DIR)

# Generate all reports
saved_files = generator.generate_all_reports(
    resort=RESORT,
    database=DATABASE,
    group_no=GROUP_NO,
    days_back=DAYS_BACK
)

print(f"\n✨ Generated {len(saved_files)} reports successfully!")


In [ ]:
# Create database connection
db = DatabaseConnection()
conn = db.connect()

# Initialize stored procedures handler
sp = StoredProcedures(conn)


In [ ]:
print("📊 Generating Revenue Report...")
revenue_df = sp.execute_revenue(DATABASE, GROUP_NO, date_ini, date_end)

if not revenue_df.empty:
    print(f"✓ Retrieved {len(revenue_df)} revenue records")
    display(revenue_df.head(10))
    print(f"\n💰 Total Revenue: ${revenue_df.iloc[:, -1].sum():,.2f}" if len(revenue_df.columns) > 0 else "")
else:
    print("⚠ No revenue data found")


In [ ]:
print("💼 Generating Payroll Report...")
payroll_df = sp.execute_payroll(RESORT, date_ini, date_end)

if not payroll_df.empty:
    print(f"✓ Retrieved {len(payroll_df)} payroll records")
    display(payroll_df.head(10))
else:
    print("⚠ No payroll data found")


In [ ]:
print("👥 Generating Visits Report...")
visits_df = sp.execute_visits(RESORT, date_ini, date_end)

if not visits_df.empty:
    print(f"✓ Retrieved {len(visits_df)} visit records")
    display(visits_df.head(10))
    if len(visits_df.columns) > 0:
        total_visits = visits_df.iloc[:, -1].sum() if len(visits_df) > 0 else 0
        print(f"\n🎿 Total Visits: {total_visits:,.0f}")
else:
    print("⚠ No visits data found")


In [ ]:
print("🌨️ Generating Weather Report...")
weather_df = sp.execute_weather(RESORT, date_ini, date_end)

if not weather_df.empty:
    print(f"✓ Retrieved {len(weather_df)} weather records")
    display(weather_df.head(10))
else:
    print("⚠ No weather data found")


In [ ]:
print("💰 Generating Complex Revenue Report with Department Breakdown...")

# Generate the complex revenue query
query = get_revenue_query(
    database=DATABASE,
    group_no=GROUP_NO,
    date_ini=date_ini.strftime('%Y-%m-%d'),
    date_end=date_end.strftime('%Y-%m-%d 23:59:59')
)

# Execute query
complex_revenue_df = execute_query_to_dataframe(conn, query)

if not complex_revenue_df.empty:
    print(f"✓ Retrieved {len(complex_revenue_df)} detailed revenue records")
    display(complex_revenue_df.head(15))
    
    if 'revenue' in complex_revenue_df.columns:
        print(f"\n💰 Total Revenue: ${complex_revenue_df['revenue'].sum():,.2f}")
        
        # Group by department if available
        if 'DepartmentTitle' in complex_revenue_df.columns:
            dept_summary = complex_revenue_df.groupby('DepartmentTitle')['revenue'].sum().sort_values(ascending=False)
            print("\n📊 Revenue by Department:")
            display(dept_summary)
else:
    print("⚠ No complex revenue data found")


In [ ]:
# Save reports as CSV files
import os

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save each report
reports = [
    (revenue_df, "revenue"),
    (payroll_df, "payroll"),
    (visits_df, "visits"),
    (weather_df, "weather"),
    (complex_revenue_df, "revenue_detailed")
]

saved_files = []
for df, report_name in reports:
    if not df.empty:
        filename = f"{RESORT}_{report_name}_{timestamp}.csv"
        filepath = os.path.join(OUTPUT_DIR, filename)
        df.to_csv(filepath, index=False)
        saved_files.append(filepath)
        print(f"✓ Saved: {filepath}")

print(f"\n📁 Saved {len(saved_files)} reports to {OUTPUT_DIR}/")


In [ ]:
# Close the database connection
db.close()
print("✓ Database connection closed")


---

## 🔍 Data Exploration

Optional: Explore the database schema and structure.


In [ ]:
print("🏔️ Available Resorts:")
for i, resort in enumerate(ResortConfig.RESORTS, 1):
    print(f"   {i}. {resort}")

print("\n📋 Purgatory Group Numbers:")
for group_no, group_name in ResortConfig.PURGATORY_GROUPS.items():
    print(f"   {group_no}: {group_name}")

print("\n📋 MCP Group Numbers:")
for group_no, group_name in ResortConfig.MCP_GROUPS.items():
    print(f"   {group_no}: {group_name}")


In [ ]:
# Uncomment to run - this queries the database schema

# from db_queries import map_db_tables_and_columns

# with DatabaseConnection() as conn:
#     tables_df = map_db_tables_and_columns(conn)
#     print(f"Found {len(tables_df)} columns across {tables_df['table'].nunique()} tables")
#     display(tables_df.head(20))
#     
#     # Show tables
#     print("\n📊 Available Tables:")
#     for table in sorted(tables_df['table'].unique()):
#         num_cols = len(tables_df[tables_df['table'] == table])
#         print(f"   • {table} ({num_cols} columns)")


In [ ]:
# Analyze revenue trends if you have the data loaded
if 'complex_revenue_df' in locals() and not complex_revenue_df.empty:
    
    print("📊 Revenue Summary Statistics:")
    if 'revenue' in complex_revenue_df.columns:
        print(f"   Total Revenue: ${complex_revenue_df['revenue'].sum():,.2f}")
        print(f"   Average per Record: ${complex_revenue_df['revenue'].mean():,.2f}")
        print(f"   Median: ${complex_revenue_df['revenue'].median():,.2f}")
        print(f"   Max: ${complex_revenue_df['revenue'].max():,.2f}")
        print(f"   Min: ${complex_revenue_df['revenue'].min():,.2f}")
        
        # Top revenue sources
        if 'account' in complex_revenue_df.columns:
            print("\n💰 Top 5 Revenue Accounts:")
            top_accounts = complex_revenue_df.nlargest(5, 'revenue')[['account', 'department', 'revenue']]
            display(top_accounts)
else:
    print("ℹ️ Run the revenue report cells above first to perform analysis")
